In [1]:
import pandas as pd
from rdflib import Graph, URIRef, Literal, BNode

In [2]:
authors_df = pd.read_csv('../Data/authors_final.csv')
genres_df = pd.read_csv('../Data/genres_pages_x_books.csv')
interactions_df = pd.read_csv('../Data/interactions_final.csv')
filtered_books_df = pd.read_csv('../Data/filtered_books.csv')

In [3]:
g = Graph()
g.parse('http://localhost:7200/repositories/project_books')

HTTPError: HTTP Error 400: 

In [ ]:
NS = URIRef('https://example.com/kb#')

AUTHOR = NS['Author']
BOOK = NS['Book']
GENRE = NS['Genre']
USER = NS['User']

READ = NS['read']
RATED = NS['rated']
REVIEWED = NS['reviewed']
BELONGS_TO = NS['belongsTo']
WRITTEN_BY = NS['writtenBy']
SIMILAR_TO = NS['similarTo']


In [ ]:
for author in authors_df.itertuples():
    author_uri = BNode()
    g.add((author_uri, RDF.TYPE, AUTHOR))
    g.add((author_uri, NS['name'], Literal(author.name)))
    g.add((author_uri, NS['averageRating'], Literal(author.average_rating)))
    g.add((author_uri, NS['ratingsCount'], Literal(author.ratings_count)))

for book in filtered_books_df.itertuples():
    book_uri = BNode()
    g.add((book_uri, RDF.TYPE, BOOK))
    g.add((book_uri, NS['title'], Literal(book.title)))
    g.add((book_uri, NS['publicationYear'], Literal(book.publication_year)))

    for author_name in book.authors:
        author_uri = URIRef(f'https://example.com/kb#Author/{author_name}')
        g.add((book_uri, WRITTEN_BY, author_uri))

    for similar_book_title in book.similar_books:
        similar_book_uri = BNode()
        g.add((similar_book_uri, RDF.TYPE, BOOK))
        g.add((similar_book_uri, NS['title'], Literal(similar_book_title)))
        g.add((book_uri, SIMILAR_TO, similar_book_uri))

for genre in genres_df.itertuples():
    genre_uri = URIRef(f'https://example.com/kb#Genre/{genre.name}')
    g.add((genre_uri, RDF.TYPE, GENRE))
    g.add((genre_uri, NS['probability'], Literal(genre[genre.name])))

    book_uri = URIRef(f'https://example.com/kb#Book/{genre.book_id}')
    g.add((book_uri, BELONGS_TO, genre_uri))

for interaction in interactions_df.itertuples():
    user_uri = BNode()
    g.add((user_uri, RDF.TYPE, USER))
    g.add((user_uri, NS['id'], Literal(interaction.user_id)))

    book_uri = URIRef(f'https://example.com/kb#Book/{interaction.book_id}')
    g.add((user_uri, READ, book_uri) if interaction.is_read else None)
    g.add((user_uri, RATED, book_uri) if interaction.rating else None)
    g.add((user_uri, REVIEWED, book_uri) if interaction.is_reviewed else None)
